add tile region

In [ ]:
DEBUG = False

In [ ]:
import os, random, glob, cv2, gc
from termcolor import colored
import sys
import time

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split

import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

import albumentations as A

from functools import partial
import scipy as sp

from PIL import Image, ImageChops

import numpy as np
import pandas as pd
import skimage.io
from tqdm.notebook import tqdm
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import albumentations as A

In [ ]:
Image.MAX_IMAGE_PIXELS = 933120000

In [ ]:
GRAY_TR = 235
N_TR = 0.85
WHITE_TR = 0.95

V = 4 # efn version (0-7)
REGION = 0 # cut region (0-3)

class config:
    SZ = 256    # not used, just to declare start point, 
    N = 36      # not used, just to declare start point, 
    LEVEL = 1   # not used, just to declare start point, 
    
    IMG_SIZE = 800
    BS = 4
    SEED = 2020
    LR = 0.0001
    LOG = f'./efnb{V}-log.txt'

In [ ]:
# seed

def seed_everything(seed=2020):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(config.SEED)


In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

# Data

In [ ]:
df = pd.read_csv('../input/pandadf/train.csv')
df = df.iloc[:1000] if DEBUG else df

In [ ]:
# shuffle
df = shuffle(df)

# split
train_df = df[df.split != 0]
val_df = df[df.split == 0]
n = len(val_df) // 2
test_df = val_df.iloc[:n]
val_df = val_df.iloc[n:]


len(train_df), len(val_df), len(test_df)

In [ ]:
train_df.head()

# Augmentation

In [ ]:
train_transforms = A.Compose([A.Transpose(p=0.5), 
                             A.VerticalFlip(p=0.5),
                             A.HorizontalFlip(p=0.5)])
val_transforms = A.Compose([])

# Dataset

In [ ]:
def get_tiles(img, SZ=256, N=36):
    '''
    dynamically select SZ and N
    img: numpy array since this image has been processed by select_level, remove_gray, crop, and transpose
    Return 
        tiles, SZ, N
    '''
    
    # pad img
    h, w, c = img.shape
    pad_h = (SZ - h % SZ) % SZ 
    pad_w = (SZ - w % SZ) % SZ 

    img2 = np.pad(img, [[pad_h // 2, pad_h - pad_h // 2], 
                        [pad_w // 2, pad_w - pad_w//2], 
                        [0,0]], constant_values=255)
    
    # choose tiles
    img3 = img2.reshape(
        img2.shape[0] // SZ,
        SZ,
        img2.shape[1] // SZ,
        SZ,
        3
    )
    
    new_row, new_col = img3.shape[0], img3.shape[2]
    img3 = img3.transpose(0, 2, 1, 3, 4).reshape(-1, SZ, SZ, 3) # (783, 256, 256, 3)
    info = (img3.reshape(img3.shape[0],-1).sum(-1) < WHITE_TR * SZ*SZ*3*255).sum() # how many tiles are not white
    
    # get new N
    possible_N = int(np.sqrt(info))**2
    if N < possible_N:
        N = possible_N
        
    idxs = np.argsort(img3.reshape(img3.shape[0],-1).sum(-1))[:N]
    tiles = img3[idxs]
        
    return tiles, SZ, N, info

In [ ]:
def find_tiles(img, SZ=256, N=36):
    tiles, SZ, N, info = get_tiles(img, SZ=SZ, N=N)

    
    # too much white tiles
    while info <= int(N_TR * N) and SZ > 64:
        SZ = SZ // 2
        tiles, SZ, N, info = get_tiles(img, SZ=SZ, N=N)
        
        
    # pad 
    # for example 32 < 30(0.85*36) 
    if tiles.shape[0] < N:
        tiles = np.pad(tiles, [ [0, N-len(tiles)], [0,0],[0,0],[0,0]], constant_values=255)
        
        
    return tiles, SZ, N, info

In [ ]:
class MyTrainDataset(Dataset):
    def __init__(self, df, split='train', shuffle_df=False, shuffle_tiles=False, region=0):
        super().__init__()
        
        if shuffle_df:
            df = shuffle(df)
        self.df = df.reset_index(drop=True)
        
        self.split = split
        self.shuffle_tiles = shuffle_tiles
        
        self.region = region
            
        
    def __len__(self):
        return len(self.df)
    
    
    def __getitem__(self, idx):
        
        # read img
        name = self.df.image_id[idx] 
        folder = self.df.folder[idx]
        path = f'../input/pandacroppedlevel1{folder}/train{folder}/{name}.jpeg'
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)        


        tiles, SZ, N, info = find_tiles(img) # find suitable tiles, SZ, N
        

        # apply transform to each img
        imgs = []
        for t in tiles:
            if self.split == 'train':
                t_aug = train_transforms(**{'image': t})['image']
            elif self.split == 'val':
                t_aug = val_transforms(**{'image': t})['image']
                
            imgs.append(t_aug) 
        
        
        # shuffle tiles
        if self.shuffle_tiles:
            imgs = shuffle(imgs)
     
        # concat
        n = int(np.sqrt(N)) # new Z
        images = np.zeros((SZ*n, SZ*n, 3), dtype=np.int32) # new SZ
        for i in range(n):
            for j in range(n):
                images[i*SZ : (i+1)*SZ, j*SZ : (j+1)*SZ, :] = imgs[i*n+j]
        
            
        # normalize 
        images = 255 - images # reverse 
        if self.split == 'train':
            images = train_transforms(image=images)['image']
        images = A.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])(image=images)['image']
        
        # get region, 4 in total
        images = cv2.resize(images, (2*config.IMG_SIZE, 2*config.IMG_SIZE))
        
        if self.region == 0: # left up
            images = images[:800, :800, :]
        elif self.region == 1:  # right up
            images = images[:800, 800:, :]
        elif self.region == 1:
            images = images[800:, :800, :]
        elif self.region == 1:
            images = images[800:, 800:, :]
    
        images = torch.tensor(images).permute(2, 0, 1)
        
        label = torch.tensor(self.df.isup_grade[idx])
        return images, label

In [ ]:
# test dataset
if DEBUG:
    train_ds = MyTrainDataset(train_df, 'train', shuffle_tiles=True, region=0)
    x, y = train_ds[0]
    print(x.shape)
    plt.imshow((x*0.5+0.5).permute(1, 2, 0).numpy())
    plt.title(y)

In [ ]:
def get_data(region=0):
    train_ds = MyTrainDataset(train_df, 'train', shuffle_tiles=True, region=region)
    train_dl = DataLoader(train_ds, batch_size=config.BS, shuffle=True, drop_last=False) # use 32 means 2 images per batch, don't shuffle to preserve 

    val_ds = MyTrainDataset(val_df, 'val', shuffle_tiles=False, region=region)
    val_dl = DataLoader(val_ds, batch_size=config.BS, shuffle=False, drop_last=False)

    test_ds = MyTrainDataset(test_df, 'val', shuffle_tiles=False, region=region)
    test_dl = DataLoader(test_ds, batch_size=config.BS, shuffle=False, drop_last=False)
    
    return train_dl, val_dl, test_dl

In [ ]:
train_dl, val_dl, test_dl = get_data(0)

In [ ]:
# test dl
if DEBUG:
    x, y = next(iter(train_dl))
    plt.imshow((x[0]*0.5+0.5).permute(1, 2, 0).numpy())
    plt.title(y[0])

# Model

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
from efficientnet_pytorch import EfficientNet

class MyModel(nn.Module):
    def __init__(self, backbone=f'efficientnet-b{V}'):
        super().__init__()
        
        self.base = EfficientNet.from_pretrained(backbone)
        self.fc = nn.Linear(self.base._fc.in_features, 1)
        self.base._fc = nn.Identity()
    
    def forward(self, x):
        x = self.base(x)
        x = self.fc(x)
        return x

In [ ]:
if DEBUG:
    model = MyModel()
    x = torch.randn(2, 3, 800, 800) # for b4, it can only accepts 512
    y = model(x)
    print(y.shape)

In [ ]:
model = MyModel()
model = model.to(device)
model.load_state_dict(torch.load('../input/pandaefnb4region/efnb4-0-1-0.8114.pth', map_location=device))

optimizer = optim.Adam(model.parameters(), lr=config.LR)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.2, patience=5, verbose=1)

In [ ]:
# freeze 
# def freeze_until(model, name):
#     flag = False
#     for n, p in model.named_parameters():
#         if n == name:
#             flag = True
#         p.requires_grad = flag
# freeze_until(model, 'model.layer4.2.conv1.weight')

# Training

In [ ]:
class OptimizedRounder():
    def __init__(self):
        self.coef_ = [0.5, 1.5, 2.5, 3.5, 4.5]

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 4
            else:
                X_p[i] = 5

        ll = cohen_kappa_score(y, X_p, weights='quadratic')

        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5, 4.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef=[0.5, 1.5, 2.5, 3.5, 4.5]):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            elif pred >= coef[3] and pred < coef[4]:
                X_p[i] = 4
            else:
                X_p[i] = 5
        return X_p

    def coefficients(self):
        '''use after self.fit or error throws'''
        return self.coef_['x']

In [ ]:
rounder = OptimizedRounder()

In [ ]:
def train_on(epoch, train_dl):
    
    torch.cuda.empty_cache()
    gc.collect()
    
    model.train()
    
    loss_epoch = []
    preds_epoch = []
    y_epoch = []
    bar = tqdm(enumerate(train_dl), total=len(train_dl))
    for i, (x, y) in bar:
        x = x.to(device, dtype=torch.float32)
        y = y.to(device, dtype=torch.float32)

        y_preds = model(x) # [batch, 1]
        y_preds = y_preds.view(-1) #[batch,]
        
        # get metrics
        loss = nn.MSELoss()(y_preds, y)

        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # add
        y_np = y.cpu().detach().numpy()
        y_preds_np = rounder.predict(y_preds.cpu().detach().numpy())
        loss_np = loss.cpu().detach().numpy()
        
        # add
        preds_epoch.append(y_preds_np)
        y_epoch.append(y_np)
        loss_epoch.append(loss_np)
    
        
        c = cohen_kappa_score(y_np, y_preds_np, weights='quadratic')
        bar.set_description('loss: %.4f, cohen: %.4f' % (loss_np, c))
        
            
        # clean
        del x, y, y_preds, loss, y_np, y_preds_np, loss_np
        torch.cuda.empty_cache()
        gc.collect()
        
        
    cohen = cohen_kappa_score(np.concatenate(preds_epoch), 
                              np.concatenate(y_epoch), 
                              weights='quadratic')
    
    print('Epoch: %d, Loss: %.4f, Cohen: %.4f' % (epoch, np.mean(loss_epoch), cohen))
    return np.mean(loss_epoch), cohen


In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
if DEBUG:
    train_on(1, train_dl)

In [ ]:
def val_on(epoch, dl):

    torch.cuda.empty_cache()
    gc.collect()
    
    model.eval()
    
    loss_epoch = []
    preds_epoch = []
    y_epoch = []
    bar = tqdm(enumerate(dl), total=len(dl))
    with torch.no_grad():
        for i, (x, y) in bar:
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.float32)
            
            y_preds = model(x).view(-1)

            # get metrics
            loss = nn.MSELoss()(y_preds, y)

            # add
            y_preds_np = y_preds.cpu().detach().numpy()
            y_np = y.cpu().detach().numpy()
            loss_np = loss.cpu().detach().numpy()
            
            loss_epoch.append(loss_np)
            preds_epoch.append(y_preds_np)
            y_epoch.append(y_np)
            
            
            # get cohen
            c = cohen_kappa_score(rounder.predict(y_preds_np), y_np, weights='quadratic')
            bar.set_description('loss: %.4f, cohen: %.4f' % (loss_np, c))

        
            # clean
            del x, y, y_preds, loss, y_np, y_preds_np, loss_np, c
            torch.cuda.empty_cache()
            gc.collect()
            

    # cohen
    preds_epoch = np.concatenate(preds_epoch)
    y_epoch = np.concatenate(y_epoch)
    
    rounder.fit(preds_epoch, y_epoch)
    coef = rounder.coefficients()
    preds_epoch = rounder.predict(preds_epoch, coef)
    cohen = cohen_kappa_score(preds_epoch, y_epoch, weights='quadratic')
    
    print('Epoch: %d, Loss: %.4f, Cohen: %.4f' % (epoch, np.mean(loss_epoch), cohen))

    return np.mean(loss_epoch), cohen, coef
               

In [ ]:
if DEBUG:
    val_on(1, val_dl)

In [ ]:
# should be put outside 
best_cohen = [0.8114] # last best score

In [ ]:
def train(region, epochs, train_dl, val_dl):
    
    for e in range(epochs):
        train_loss, train_cohen = train_on(e, train_dl)
        val_loss, val_cohen, coef = val_on(e, val_dl)
        
        # adjust lr
        scheduler.step(val_loss)

        # write to log
        with open(config.LOG, 'a') as f:
            f.write(time.ctime() + f' Epoch: {e}, Train loss: {(train_loss):.4f}, Train cohen: {(train_cohen):.4f}\n   Val loss: {(val_loss):.4f}, Val cohen:{(val_cohen):.4f}, Coef: {coef} \n\n')
            
        # save best
        best_cohen.append(val_cohen)
        if val_cohen >= max(best_cohen):
            print('save best model')
            torch.save(model.state_dict(), f'efnb{V}-{region}-best.pth')
            
        torch.save(model.state_dict(), f'./efnb{V}-{region}-{e}.pth')
        
    torch.save(model.state_dict(), f'./efnb{V}-{region}-final.pth')

In [ ]:
torch.cuda.empty_cache()
gc.collect()

In [ ]:
train_dl, val_dl, test_dl = get_data(region=0)
train(0, 10, train_dl, val_dl)

In [ ]:
# model.load_state_dict(torch.load(f'efnb{V}-6.pth'))

In [ ]:
val_on(1, test_dl) 
# 0.8858
# [0.51885345, 1.53662393, 2.44524873, 3.61225587, 4.29971003]